# Algorithms: low-level interface
Once we have data for GST, there are several *algorithms* we can run on it to produce tomographic estimates. Depending on the amount of data you have, and time available for running Gate Set Tomography, one algorithm may be preferable over the others.  **What is typically thought of as "standard GST" is the iterative maximum-likelihood optimization implemented by `do_iterative_mlgst`** which uses a combination of minimum-$\chi^2$ GST and maximum-likelihood GST.

`pygsti` provides support for the following "primary" GST algorithms:

* **Linear Gate Set Tomography (LGST)**: Uses short gate sequences to quickly compute a rough (low accuracy) estimate of a gate set by linear inversion.

* **Extended Linear Gate Set Tomography (eLGST or EXLGST)**: Minimizes the sub-of-squared errors between independent LGST estimates and the estimates obtained from a single gate set to find a best-estimate gate set.  This is typically done in an interative fashion, using LGST estimates for longer and longer sequences. 

* **Minimum-$\chi^2$ Gate Set Tomography (MC2GST)**: Minimizes the $\chi^{2}$ statistic of the data frequencies and gate set probabilities to find a best-estimate gate set.  Typically done in an interative fashion, using successively larger sets of longer and longer gate sequences. 

* **Maximum-Likelihood Gate Set Tomography (MLGST)**: Maximizes the log-likelihood statistic of the data frequencies and gate set probabilities to find a best-estimate gate set.  Typically done in an interative fashion similar to MC2GST.  This maximum likelihood estimation (MLE) is very well-motivated from a statistics standpoint and should be the **most accurate among the algorithms**. 

If you're curious, the implementation of the algorithms for LGST, EXLGST, MC2GST, and MLGST may be found in the `pygsti.algorithms.core` module.  In this tutorial, we'll show how to invoke each of these algorithms.

Additionally, `pygsti` contains **gauge-optimization** algorithms.  Because the outcome data (the input to the GST algorithms above) only determines a gate set up to some number of un-physical "gauge" degrees of freedom, it is often desirable to optimize the `GateSet` estimate obtained from a GST algorithm within the space of its gauge freedoms.  This process is called "gauge-optimization" and the final part of this tutorial demonstrates how to gauge-optimize a gate set using various criteria.

##  Primary GST Algorithms
The ingredients needed to as input to the "primary" GST algorithms are:
- a "target" `GateSet` which defines the desired gates.  This gate set is used by LGST to specify the various gate, state preparation, POVM effect, and SPAM labels, as well as to provide an initial guess for the *gauge* degrees of freedom.
- a `DataSet` containing the data that GST attempts to fit using the probabilities generated by a single `GateSet`.  This data set must at least contain the data for the gate sequences required by the algorithm that is chosen.
- for EXLGST, MC2GST, and MLGST, a list-of-lists of `GateString` objects, which specify which gate strings are used during each iteration of the algorithm (the length of the top-level list defines the number of interations).  Note that which gate strings are included in these lists is different for EXLGST than it is for MC2GST and MLGST.

In [1]:
from __future__ import print_function

In [2]:
import pygsti
import json

In [3]:
#We'll use the standard I, X(pi/2), Y(pi/2) gate set that we generated data for in the DataSet tutorial
from pygsti.construction import std1Q_XYI

gs_target = std1Q_XYI.gs_target
prep_fiducials = std1Q_XYI.prepStrs
meas_fiducials = std1Q_XYI.effectStrs
germs = std1Q_XYI.germs

maxLengthList = [1,2,4,8,16] #for use in iterative algorithms

ds = pygsti.io.load_dataset("tutorial_files/Example_Dataset.txt", cache=True)
dsLowCounts = pygsti.io.load_dataset("tutorial_files/Example_Dataset_LowCnts.txt", cache=True)

depol_gateset = gs_target.depolarize(gate_noise=0.1)

print("Loaded target gateset with gate labels: ", gs_target.gates.keys())
print("Loaded fiducial lists of lengths: ", (len(prep_fiducials),len(meas_fiducials)))
print("Loaded dataset of length: ", len(ds))

Loading from cache file: tutorial_files/Example_Dataset.txt.cache
Loading from cache file: tutorial_files/Example_Dataset_LowCnts.txt.cache
Loaded target gateset with gate labels:  odict_keys(['Gi', 'Gx', 'Gy'])
Loaded fiducial lists of lengths:  (6, 6)
Loaded dataset of length:  3382


### Using LGST to get an initial estimate

An important and distinguising property of the LGST algorithm is that it does *not* require an initial-guess `GateSet` as an input.  It uses linear inversion and short sequences to obtain a rough gate set estimate.  As such, it is very common to use the LGST estimate as the initial-guess starting point for more advanced forms of GST.

In [4]:
#Run LGST to get an initial estimate for the gates in gs_target based on the data in ds

#run LGST
gs_lgst = pygsti.do_lgst(ds, prep_fiducials, meas_fiducials, targetGateset=gs_target, verbosity=1)

#Gauge optimize the result to match the target gateset
gs_lgst_after_gauge_opt = pygsti.gaugeopt_to_target(gs_lgst, gs_target, verbosity=1)

#Contract the result to CPTP, guaranteeing that the gates are CPTP
gs_clgst = pygsti.contract(gs_lgst_after_gauge_opt, "CPTP", verbosity=1)

--- LGST ---
Gauge optimization completed in 0.016434s.
--- Contract to CPTP (direct) ---
The closest legal point found was distance: 0.000146798833766


In [5]:
print(gs_lgst)

rho0 =    0.7070  -0.0210   0.0312   0.7482


Mdefault = POVM with effect vectors:
0:
 0.73
   0
   0
 0.65

1:
 0.69
   0
   0
-0.65



Gi = 
   0.9999        0   0.0001        0
  -0.0149   0.9281   0.0187   0.0254
   0.0192   0.0090   0.9016   0.0196
  -0.0081   0.0103   0.0014   0.9060


Gx = 
   1.0000        0        0        0
  -0.0056   0.9081  -0.0058  -0.0006
  -0.0085   0.0158  -0.0434  -0.9973
  -0.0584   0.0151   0.8106   0.0057


Gy = 
   1.0000        0  -0.0002        0
   0.0102  -0.0238   0.0347   0.9867
   0.0099  -0.0169   0.8902  -0.0312
  -0.0648  -0.8111  -0.0071   0.0178






### Extended LGST (eLGST or EXLGST)
EXLGST requires a list-of-lists of gate strings, one per iteration.  The elements of these lists are typically repetitions of short "germ" strings such that the final strings does not exceed some maximum length.  We created such lists in the gate string tutorial.  Now, we just load these lists from the text files they were saved in.

In [6]:
#Get rho and E specifiers, needed by LGST
elgstListOfLists = pygsti.construction.make_elgst_lists(gs_target, germs, maxLengthList)
           
#run EXLGST.  The result, gs_exlgst, is a GateSet containing the estimated quantities
gs_exlgst = pygsti.do_iterative_exlgst(ds, gs_clgst, prep_fiducials, meas_fiducials, elgstListOfLists,
                                       targetGateset=gs_target, verbosity=2)

--- Iterative eLGST:  Iter 5 of 5 ; 43 gate strings ---: 


### Minimum-$\chi^2$ GST (MC2GST)
MC2GST and MLGST also require a list-of-lists of gate strings, one per iteration.  However, the elements of these lists are typically repetitions of short "germ" strings *sandwiched between fiducial strings* such that the repeated-germ part of the string does not exceed some maximum length.  We created such lists in the gate string tutorial.  Now, we just load these lists from the text files they were saved in.

In [7]:
#Get lists of gate strings for successive iterations of LSGST to use
specs = pygsti.construction.build_spam_specs(prepStrs=prep_fiducials, effectStrs=meas_fiducials)
lsgstListOfLists = pygsti.construction.make_lsgst_lists(gs_target, prep_fiducials, meas_fiducials, germs, maxLengthList)
  
#run MC2GST.  The result is a GateSet containing the estimated quantities
gs_mc2 = pygsti.do_iterative_mc2gst(ds, gs_clgst, lsgstListOfLists, verbosity=2)

--- Iterative MC2GST: Iter 1 of 5  92 gate strings ---: 
  --- Minimum Chi^2 GST ---
  Sum of Chi^2 = 59.4319 (92 data params - 44 model params = expected mean of 48; p-value = 0.124651)
  Completed in 0.1s
      Iteration 1 took 0.1s
  
--- Iterative MC2GST: Iter 2 of 5  168 gate strings ---: 
  --- Minimum Chi^2 GST ---
  Sum of Chi^2 = 121.138 (168 data params - 44 model params = expected mean of 124; p-value = 0.555947)
  Completed in 0.1s
      Iteration 2 took 0.1s
  
--- Iterative MC2GST: Iter 3 of 5  450 gate strings ---: 
  --- Minimum Chi^2 GST ---
  Sum of Chi^2 = 420.911 (450 data params - 44 model params = expected mean of 406; p-value = 0.294486)
  Completed in 0.3s
      Iteration 3 took 0.3s
  
--- Iterative MC2GST: Iter 4 of 5  862 gate strings ---: 
  --- Minimum Chi^2 GST ---
  Sum of Chi^2 = 802.789 (862 data params - 44 model params = expected mean of 818; p-value = 0.641289)
  Completed in 0.5s
      Iteration 4 took 0.5s
  
--- Iterative MC2GST: Iter 5 of 5  1282

In [8]:
#Write the resulting EXLGST and MC2GST results to gate set text files for later reference.
pygsti.io.write_gateset(gs_exlgst, "tutorial_files/Example_eLGST_Gateset.txt","# Example result from running eLGST")
pygsti.io.write_gateset(gs_mc2,  "tutorial_files/Example_MC2GST_Gateset.txt","# Example result from running MC2GST")

In [9]:
#Run MC2GST again but use a DataSet with a lower number of counts 
gs_mc2_lowcnts = pygsti.do_iterative_mc2gst(dsLowCounts, gs_clgst, lsgstListOfLists, verbosity=2)

--- Iterative MC2GST: Iter 1 of 5  92 gate strings ---: 
  --- Minimum Chi^2 GST ---
  Sum of Chi^2 = 66.3177 (92 data params - 44 model params = expected mean of 48; p-value = 0.0409368)
  Completed in 0.1s
      Iteration 1 took 0.1s
  
--- Iterative MC2GST: Iter 2 of 5  168 gate strings ---: 
  --- Minimum Chi^2 GST ---
  Sum of Chi^2 = 143.415 (168 data params - 44 model params = expected mean of 124; p-value = 0.112061)
  Completed in 0.2s
      Iteration 2 took 0.2s
  
--- Iterative MC2GST: Iter 3 of 5  450 gate strings ---: 
  --- Minimum Chi^2 GST ---
  Sum of Chi^2 = 425.239 (450 data params - 44 model params = expected mean of 406; p-value = 0.245704)
  Completed in 0.3s
      Iteration 3 took 0.3s
  
--- Iterative MC2GST: Iter 4 of 5  862 gate strings ---: 
  --- Minimum Chi^2 GST ---
  Sum of Chi^2 = 830.538 (862 data params - 44 model params = expected mean of 818; p-value = 0.37264)
  Completed in 1.1s
      Iteration 4 took 1.1s
  
--- Iterative MC2GST: Iter 5 of 5  1282

### Maximum Likelihood GST (MLGST)
Executing MLGST is very similar to MC2GST: the same gate string lists can be used and calling syntax is nearly identitcal.

In [10]:
#run MLGST.  The result is a GateSet containing the estimated quantities
gs_mle = pygsti.do_iterative_mlgst(ds, gs_clgst, lsgstListOfLists, verbosity=2)

--- Iterative MLGST: Iter 1 of 5  92 gate strings ---: 
  --- Minimum Chi^2 GST ---
  Sum of Chi^2 = 59.4319 (92 data params - 44 model params = expected mean of 48; p-value = 0.124651)
  Completed in 0.1s
  2*Delta(log(L)) = 59.705
  Iteration 1 took 0.1s
  
--- Iterative MLGST: Iter 2 of 5  168 gate strings ---: 
  --- Minimum Chi^2 GST ---
  Sum of Chi^2 = 121.138 (168 data params - 44 model params = expected mean of 124; p-value = 0.555947)
  Completed in 0.1s
  2*Delta(log(L)) = 121.606
  Iteration 2 took 0.2s
  
--- Iterative MLGST: Iter 3 of 5  450 gate strings ---: 
  --- Minimum Chi^2 GST ---
  Sum of Chi^2 = 420.911 (450 data params - 44 model params = expected mean of 406; p-value = 0.294486)
  Completed in 0.3s
  2*Delta(log(L)) = 421.628
  Iteration 3 took 0.5s
  
--- Iterative MLGST: Iter 4 of 5  862 gate strings ---: 
  --- Minimum Chi^2 GST ---
  Sum of Chi^2 = 802.789 (862 data params - 44 model params = expected mean of 818; p-value = 0.641289)
  Completed in 0.6s
  2

In [11]:
#Run MLGST again but use a DataSet with a lower number of counts 
gs_mle_lowcnts = pygsti.do_iterative_mlgst(dsLowCounts, gs_clgst, lsgstListOfLists, verbosity=2)

--- Iterative MLGST: Iter 1 of 5  92 gate strings ---: 
  --- Minimum Chi^2 GST ---
  Sum of Chi^2 = 66.3177 (92 data params - 44 model params = expected mean of 48; p-value = 0.0409368)
  Completed in 0.1s
  2*Delta(log(L)) = 67.6025
  Iteration 1 took 0.2s
  
--- Iterative MLGST: Iter 2 of 5  168 gate strings ---: 
  --- Minimum Chi^2 GST ---
  Sum of Chi^2 = 143.415 (168 data params - 44 model params = expected mean of 124; p-value = 0.112061)
  Completed in 0.2s
  2*Delta(log(L)) = 147.233
  Iteration 2 took 0.3s
  
--- Iterative MLGST: Iter 3 of 5  450 gate strings ---: 
  --- Minimum Chi^2 GST ---
  Sum of Chi^2 = 425.239 (450 data params - 44 model params = expected mean of 406; p-value = 0.245704)
  Completed in 0.4s
  2*Delta(log(L)) = 436.372
  Iteration 3 took 0.7s
  
--- Iterative MLGST: Iter 4 of 5  862 gate strings ---: 
  --- Minimum Chi^2 GST ---
  Sum of Chi^2 = 830.538 (862 data params - 44 model params = expected mean of 818; p-value = 0.37264)
  Completed in 1.6s
  

## Gauge Optimization
All gauge optimization algorithms perform essentially the same task - to find the gate set which optimizes some objective function from within the set or space of gate sets that are gauge-equivalent to some starting set.  This is accomplished in `pygsti` using the following mechanism:
- one begins with an initial `GateSet`, call it `gs`, to be gauge-optimized.
- a `pygsti.objects.GaugeGroup` instance defines a parameterized group of allowable gauge transformations.  This gauge group must be compatible with the `gs`'s parameterization, so that `gs.transform` (which calls `Gate.transform` and `SPAMVec.transform`) is able to process elements of the `GaugeGroup` (obtained via a call to `GaugeGroup.get_element(params)`).  That is, the gauge transformation must map between gatesets with the *same* parameterization (that give by `gs`).  Because of the close interplay between a gate set's parameterization and its allowed gauge transformations, `GateSet` objects can contain a `GaugeGroup` instance as their `default_gauge_group` member. In many circumstances, `gs.default_gauge_group` is set to the correct gauge group to use for a given `GateSet`.
- `pygsti.gaugeopt_custom(...)` takes an intial `GateSet`, an objective function, and a `GaugeGroup` (along with other optimization parameters) and returns a gauge-optimized `GateSet`.  Note that if its `gauge_group` argument is left as `None`, then the gate set's default gauge group is used.  And objective function which takes a single `GateSet` argument and returns a float can be supplied, giving the user a fair amount of flexiblity.
- since usually the objective function is one which compares the gate set being optimized to a fixed "target" gate set, `pygsti.gaugeopt_to_target(...)` is a routine able to perform these common types of gauge optimization.  Instead of an objective function, `gaugeopt_to_target` takes a target `GateSet` and additional arguments (see below) from which it constructs a objective function and then calls `gaugeopt_custom`.  It is essetially a convenience routine for constructing common gauge optimization objective functions.  Relevant arguments which affect what objective function is used are:
  - `targetGateset` : the `GateSet` to compare against - i.e., the one you want to gauge optimize toward. **Note that this doesn't have to be a set of ideal gates **- it can be any (imperfect) gate set that reflects your expectations about what the estimates should look like.
  - `itemWeights` : a dictionary of weights allowing different gates and/or SPAM operations to be weighted differently when computing the objective function's value.
  - `CPpenalty` : a prefactor multiplying the sum of all the negative Choi-matrix eigenvalues corresponding to each of the gates.
  - `TPpenalty` : a prefactor multiplying the sum of absoulte-value differences between the first row of each gate matrix and `[1 0 ... 0 ]` and the discrpance between the first element of each state preparation vector and its expected value.
  - `validSpamPenalty` : a prefactor multiplying penalty terms enforcing the non-negativity of state preparation eigenavlues and that POVM effect eigenvalues lie between 0 and 1.
  - `gatesMetric` : how to compare corresponding gates in the gauge-optimized and target sets. `"frobenius`" uses the frobenius norm (weighted before taking the final sqrt), `"fidelity"` uses the *squared process infidelity* (squared to avoid negative-infidelity issues in non-TP gate sets), and `"tracedist"` uses the trace distance (weighted after computing the trace distance between corresponding gates).
  - `spamMetric` : how to compare corresponding SPAM vectors. `"frobenius"` (the default) should be used here, as `"fidelity"` and `"tracedist"` compare the "SPAM gates" -- the outer product of state prep and POVM effect vectors -- which isn't a meaningful metric.
  
The cell below demonstrates some of common usages of `gaugeopt_to_target`.

In [12]:
gs = gs_mle.copy() #we'll use the MLGST result from above as an example
gs_go1 = pygsti.gaugeopt_to_target(gs, gs_target) # optimization to the perfect target gates
gs_go2 = pygsti.gaugeopt_to_target(gs, depol_gateset) # optimization to a "guess" at what the estimate should be
gs_go3 = pygsti.gaugeopt_to_target(gs, gs_target, {'gates': 1.0, 'spam': 0.01}) 
  # weight the gates differently from the SPAM operations
gs_go4 = pygsti.gaugeopt_to_target(gs, gs_target, {'gates': 1.0, 'spam': 0.01, 'Gx': 10.0, 'E0': 0.001}) 
  # weight an individual gate/SPAM separately (note the specific gate/SPAM labels always override
  # the more general 'gates' and 'spam' weight values). 
gs_go5 = pygsti.gaugeopt_to_target(gs, gs_target, gatesMetric="tracedist") #use trace distance instead of frobenius

print("Default gauge group = ",type(gs.default_gauge_group)) # default is FullGaugeGroup
gs_go6 = pygsti.gaugeopt_to_target(gs, gs_target, gauge_group=pygsti.objects.UnitaryGaugeGroup(gs.dim, 'pp'))
  #gauge optimize only over unitary gauge transformations

print("\ngaugeopt_to_target output:")
gs_go7 = pygsti.gaugeopt_to_target(gs, gs_target, verbosity=3) # show output
print("Final frobenius distance between gs_go7 and gs_target = ", gs_go7.frobeniusdist(gs_target))

Default gauge group =  <class 'pygsti.objects.gaugegroup.FullGaugeGroup'>

gaugeopt_to_target output:
  --- Gauge Optimization (ls method) ---
--- Outer Iter 0: norm_f = 0.0916384, mu=0
--- Outer Iter 1: norm_f = 0.0916365, mu=0.00171251
--- Outer Iter 2: norm_f = 0.0916365, mu=0.000570835
  Least squares message = Both actual and predicted relative reductions in the sum of squares are at most 1e-08
Gauge optimization completed in 0.0197349s.
Final frobenius distance between gs_go7 and gs_target =  0.0390803671329


## Compare MLGST with MC2GST (after gauge optimization)

Both MLGST and MC2GST use a $\chi^{2}$ optimization procedure for all but the final iteration. For the last set of gatestrings (the last iteration), MLGST uses a maximum likelihood estimation. Below, we show how close the two estimates are to one another.  Before making the comparison, however, we **optimize the gauge** so the estimated gates are as close to the target gates as the gauge degrees of freedom allow.

In [13]:
# We optimize over the gate set gauge
gs_mle         = pygsti.gaugeopt_to_target(gs_mle,depol_gateset)
gs_mle_lowcnts = pygsti.gaugeopt_to_target(gs_mle_lowcnts,depol_gateset)
gs_mc2         = pygsti.gaugeopt_to_target(gs_mc2,depol_gateset)
gs_mc2_lowcnts = pygsti.gaugeopt_to_target(gs_mc2_lowcnts,depol_gateset)

In [14]:
print("Frobenius diff btwn MLGST  and datagen = {0}".format(round(gs_mle.frobeniusdist(depol_gateset), 6)))
print("Frobenius diff btwn MC2GST and datagen = {0}".format(round(gs_mc2.frobeniusdist(depol_gateset), 6)))
print("Frobenius diff btwn MLGST  and LGST    = {0}".format(round(gs_mle.frobeniusdist(gs_clgst), 6)))
print("Frobenius diff btwn MLGST  and MC2GST  = {0}".format(round(gs_mle.frobeniusdist(gs_mc2), 6)))
print("Chi^2 ( MC2GST ) = {0}".format(round(pygsti.chi2(gs_mc2, ds, lsgstListOfLists[-1]), 4)))
print("Chi^2 ( MLGST )  = {0}".format(round(pygsti.chi2(gs_mle, ds, lsgstListOfLists[-1] ), 4)))
print("LogL  ( MC2GST ) = {0}".format(round(pygsti.logl(gs_mc2, ds, lsgstListOfLists[-1]), 4)))
print("LogL  ( MLGST )  = {0}".format(round(pygsti.logl(gs_mle, ds, lsgstListOfLists[-1]), 4)))

Frobenius diff btwn MLGST  and datagen = 0.002548
Frobenius diff btwn MC2GST and datagen = 0.00255
Frobenius diff btwn MLGST  and LGST    = 0.008528
Frobenius diff btwn MLGST  and MC2GST  = 5.3e-05
Chi^2 ( MC2GST ) = 1242.5172
Chi^2 ( MLGST )  = 1242.8786
LogL  ( MC2GST ) = -2136565.9592
LogL  ( MLGST )  = -2136565.7782


Notice that, as expected, the MC2GST estimate has a *slightly* lower $\chi^{2}$ score than the MLGST estimate, and the MLGST estimate has a *slightly* higher loglikelihood than the MC2GST estimate. In addition, _both_ are close (in terms of the Frobenius difference) to the depolarized gateset. Which is good - it means GST is giving us estimates which are close to the _true_ gateset used to generate the data.  Performing the same analysis with the low-count data shows larger differences between the two, which is expected since the $\chi^2$ and loglikelihood statistics are more similar at large $N$, that is, for large numbers of samples.

In [15]:
print("LOW COUNT DATA:")
print("Frobenius diff btwn MLGST  and datagen = {0}".format(round(gs_mle_lowcnts.frobeniusdist(depol_gateset), 6)))
print("Frobenius diff btwn MC2GST and datagen = {0}".format(round(gs_mc2_lowcnts.frobeniusdist(depol_gateset), 6)))
print("Frobenius diff btwn MLGST  and LGST    = {0}".format(round(gs_mle_lowcnts.frobeniusdist(gs_clgst), 6)))
print("Frobenius diff btwn MLGST  and MC2GST  = {0}".format(round(gs_mle_lowcnts.frobeniusdist(gs_mc2), 6)))
print("Chi^2 ( MC2GST )  = {0}".format(round(pygsti.chi2(gs_mc2_lowcnts, dsLowCounts, lsgstListOfLists[-1]), 4)))
print("Chi^2 ( MLGST )   = {0}".format(round(pygsti.chi2(gs_mle_lowcnts, dsLowCounts, lsgstListOfLists[-1] ), 4)))
print("LogL  ( MC2GST )  = {0}".format(round(pygsti.logl(gs_mc2_lowcnts, dsLowCounts, lsgstListOfLists[-1]), 4)))
print("LogL  ( MLGST )   = {0}".format(round(pygsti.logl(gs_mle_lowcnts, dsLowCounts, lsgstListOfLists[-1]), 4)))

LOW COUNT DATA:
Frobenius diff btwn MLGST  and datagen = 0.010385
Frobenius diff btwn MC2GST and datagen = 0.010475
Frobenius diff btwn MLGST  and LGST    = 0.014462
Frobenius diff btwn MLGST  and MC2GST  = 0.011165
Chi^2 ( MC2GST )  = 1257.4892
Chi^2 ( MLGST )   = 1265.1072
LogL  ( MC2GST )  = -106884.3736
LogL  ( MLGST )   = -106880.5702
